# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# HW4

### import the necessary library

In [1]:
import json
import datetime
from collections import Counter
import random

In [2]:
# For measuring the total running time of this notebook
start_notebook_t = datetime.datetime.now()
print(start_notebook_t)

2024-02-04 14:56:06.704986


In [3]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


24/02/04 14:56:08 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.137.8 instead (on interface enp48s0)
24/02/04 14:56:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 14:56:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/04 14:56:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Read data

In [4]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [5]:
# tweets_rdd = tweets_rdd.sample(False, 0.1, seed=42)


### Show a sample of tweets

In [6]:
tweets_rdd.takeSample(False, 1, 19)

['{"in_reply_to_status_id_str": null, "in_reply_to_user_id_str": null, "in_reply_to_status_id": null, "in_reply_to_user_id": null, "in_reply_to_screen_name": null, "truncated": false, "is_quote_status": false, "retweet_count": 3, "reply_count": 0, "quote_count": 0, "favorite_count": 0, "favorited": false, "retweeted": true, "possibly_sensitive": null, "lang": "fa", "geo": null, "view_count": 0, "tweet_type": "retweeted", "emojis": null, "text": "مردک وطن\u200cفروش بدبخت، ترانه آفتابکاران جنگل مال چی بوده که خوندن؟! موسیقی اصیل ایرانی بوده؟! یا سرود تروریست\u200cهای چریک کمونیست بوده؟!\\nاشکال نداره، شما به خزعبل گویی ادامه بدید. خوبیش اینه که مردم دارن کم کم متوجه میشن که شنا دگوری\u200cهای خائن وطن\u200cفروش چه جماعتی هستید.\\n\\n#ننگ_بر_سه_فاسد_ملا_چپی_مجاهد", "user": {"id": "315689091", "id_str": "315689091", "created_at": 1307868062, "name": "The Nerd", "screen_name": "NewbieBadASS", "location": "خیابون هفده شهریور ", "url": "https://t.co/ajpmxoYoTe", "description": "A supporter of

### Show size of data (No. tweets)

In [7]:
tweets_rdd.count()

4175860

## User Recommender 

In [8]:
start_user_recommending_t = datetime.datetime.now()
print(start_user_recommending_t)


2024-02-04 14:56:49.636752


### Define functions for extract relations graph

In [9]:
def to_tid_user(tid, uid, origin_tid, origin_uid):
    ''' 
    This function get some information of a tweet and Returns
    the two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    related to this tweet (FOR EACH TWEET).

    The output is a list of tuples, where each tuple contains
    a tweet ID and a set of users relate to it.
    Output format: [(tid1, {u1, u2}), (tid2, {u1, u2})]
    '''
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return []

    return [(tid, {uid, origin_uid[0]})]

def extract_tuples(tweet_info):
    '''
    This function extract some information of tweet and send it
    to to_tid_user funcions and return its results
    '''
    return to_tid_user(
        tweet_info.get('id'),
        tweet_info["user"].get('id'),
        [
            tweet_info.get('in_reply_to_status_id_str', None),
            tweet_info.get('quoted_status', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('id'),
        ],
        [
            tweet_info.get('in_reply_to_user_id_str', None),
            tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
        ],
    )


def extract_relations(x):
    '''
    This function extracts the MAX_REALTION relationships
    between users who interacted with this tweet.
    
    x contains the tweet_id (tid) and a set of user_id.
    x: (tid, {u1, u2, ...})

    The output is a list that each element of it, contains the user_id
    (u1) and a set if user_ids that has relation to u1 (called uj).
    output: 
    [(u1, {uj}), (u2, {uj}), ]
    
    '''
    MAX_REALTION = 10
    _, users = x
    relations = []
    for u in list(users):
        u_users = users.copy()
        u_users.remove(u)
        u_users = set(random.choices(list(u_users), k=min(MAX_REALTION, len(u_users))))
        relations.append((u, u_users))

        
    return relations



### Extracting the ralations graph


In [10]:
# [(tid, {u1,u2}), ...]
tid_user_score_rdd = tweets_rdd.flatMap(lambda x: extract_tuples(json.loads(x)))

# [(tid, {u1, u2, u3, ...}), ...]
tid_users_scores_rdd = tid_user_score_rdd.reduceByKey(lambda x, y: x.union(y))

# [(u1, {u2, u3, ...})]
user_relations_rdd = tid_users_scores_rdd.flatMap(extract_relations)

# [(u1, {u2, u3, ...})]
user_relations_rdd = user_relations_rdd.reduceByKey(lambda x, y: x.union(y))

# collect relations from rdd
users_relations = user_relations_rdd.collect()


### select a user ID

In [11]:
'''
you must choose user id in this section.
you can change # in `list(user_relations[#][1])` for this purpose.
'''
q_uid = random.choice(list(users_relations[20][1]))
# q_uid = '1726657633799569408'  # select q_uid same as HW3
q_uid = '1365385186364567553'  # select q_uid same as HW3
print('query user id:', q_uid)

query user id: 1365385186364567553


In [12]:
users_relations_dict = dict(users_relations)

### Do random walk on memory for selected user based on his/her relations

In [13]:
N = 1000  # N times start random walk from begining
MAX_WALK = 4  # Each time take 10 steps
scores = {}
for i in range(N):
    current_uid = q_uid
    cnt = 0
    while cnt < MAX_WALK:
        current_user_relations = list(users_relations_dict[current_uid])
        if not current_user_relations:
            break
        next_uid = random.choice(current_user_relations)
        scores[next_uid] = scores.get(next_uid, 0) + 1
        current_uid = next_uid
        cnt += 1

In [14]:
mcru = sorted(scores.items(), key=lambda x:-x[1])[:20]  # 20 most common recommended users
if q_uid in mcru:
    mcru.remove(q_uid)
mcru_id, scores = zip(*mcru)
mcru

[('1365385186364567553', 28),
 ('1591547142136627202', 12),
 ('1718268858056736768', 10),
 ('806244423961104388', 10),
 ('1408332971262693377', 9),
 ('1682143930119323650', 8),
 ('1716147579954634752', 8),
 ('1088222529041416193', 7),
 ('1237608244673007616', 7),
 ('367663142', 7),
 ('1005209207203823616', 7),
 ('1491528861456248833', 7),
 ('1418467603337601026', 7),
 ('1557754149357355008', 7),
 ('1334075031236861952', 7),
 ('1722953366756630528', 6),
 ('827011762255958017', 6),
 ('1534201832448434179', 6),
 ('1684786665024282624', 6),
 ('1034695487784280064', 6)]

In [15]:
end_extract_recommended_users_tweets_t = datetime.datetime.now()
print(end_extract_recommended_users_tweets_t)



2024-02-04 14:57:27.328016


## Show Tweets of recommended Users

### Define functions needed to separate tweets based on tweet_type and display tweets appropriately

In [16]:
def extract_user_origin_tweets(tweets, uid):
    origin_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'generated':
            if t.get('user',{}).get('id','0') == uid:
                origin_tweets.append(t)

    return origin_tweets


def extract_user_replied_tweets(tweets, uid):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('user',{}).get('id','0') == uid:
                replied_tweets.append(t)

    return replied_tweets


def extract_user_quoted_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('user',{}).get('id','0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_user_retweeted_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('user',{}).get('id','0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def show_tweets(tweets, tweets_type):
    print('-'*100)
    print(f'Number of {tweets_type}:', len(tweets), '\n')
    ts = [t['text'].replace('\u200c', '').split('\n') for t in tweets]
    for t in ts:
        print('-'*100)
        for line in t:
            print(line)
    print('-'*100)


In [17]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                  'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                  'quoted_status', 'retweeted_status','profile_image_url',
                 ]


In [18]:
rec_users_tweets = tweets_rdd.filter(lambda x: json.loads(x)['user']['id'] in mcru_id or json.loads(x)['user']['id']==q_uid)\
                            .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                            .collect()



#### Tweets of Query User

In [19]:
print('','#'*100,'\n', 'Tweets of Query User\n', '#'*100,'\n')
show_tweets(extract_user_origin_tweets(rec_users_tweets, q_uid), 'user_origin_tweets')
show_tweets(extract_user_quoted_tweets(rec_users_tweets, q_uid), 'user_quoted_tweets')
show_tweets(extract_user_retweeted_tweets(rec_users_tweets, q_uid), 'user_retweeted_tweets')
show_tweets(extract_user_replied_tweets(rec_users_tweets, q_uid), 'user_replied_tweets')

 #################################################################################################### 
 Tweets of Query User
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 57 

----------------------------------------------------------------------------------------------------
دوستان برام بسیارغم انگیزتر و ناراحت کننده تر اینه که رهبر معظم و افرادهم فکر ایشان  که برای سیرتاپیاز مردم نظریه پردازی می کنند حتی اشاره وانتقادکوچکی درقبال هحمه ی خودخواسته ی 15 میلیون قاتل ومتجاوزبعنف ودزد #افغانستانی به مردم خودشان وکشته شدن روزانه مردم بدست اینها نکرده اند!
----------------------------------------------------------------------------------------------------
نه کوچکترین قرابت نژادی داریم!،نه کوچکترین قرابت فرهنگی!،نه کوچکترین قرابت مذهبی!،فقط هردوی مابه فارسی صحبت می کنیم همین!بنابراین رویای ترمیم بافت جمعیت

#### Tweets of Recommended User

In [20]:
for (recommended_uid, rep) in mcru:
    print('','#'*100,'\n', f'User ID: {recommended_uid}, repetitions: {rep}\n', '#'*100,'\n')
    show_tweets(extract_user_origin_tweets(rec_users_tweets, recommended_uid), 'user_origin_tweets')
    show_tweets(extract_user_quoted_tweets(rec_users_tweets, recommended_uid), 'user_quoted_tweets')
    show_tweets(extract_user_retweeted_tweets(rec_users_tweets, recommended_uid), 'user_retweeted_tweets')
    show_tweets(extract_user_replied_tweets(rec_users_tweets, recommended_uid), 'user_replied_tweets')



 #################################################################################################### 
 User ID: 1365385186364567553, repetitions: 28
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 57 

----------------------------------------------------------------------------------------------------
دوستان برام بسیارغم انگیزتر و ناراحت کننده تر اینه که رهبر معظم و افرادهم فکر ایشان  که برای سیرتاپیاز مردم نظریه پردازی می کنند حتی اشاره وانتقادکوچکی درقبال هحمه ی خودخواسته ی 15 میلیون قاتل ومتجاوزبعنف ودزد #افغانستانی به مردم خودشان وکشته شدن روزانه مردم بدست اینها نکرده اند!
----------------------------------------------------------------------------------------------------
نه کوچکترین قرابت نژادی داریم!،نه کوچکترین قرابت فرهنگی!،نه کوچکترین قرابت مذهبی!،فقط هردوی مابه فارسی صحبت می کنیم همین!بنابرا

In [21]:
end_notebook_t = datetime.datetime.now()
print(end_notebook_t,'\n')

2024-02-04 14:57:56.495482 



In [22]:
print("The total time spent for showing a sample of tweets and count tweets:      ", str(start_user_recommending_t - start_notebook_t).split('.')[0])
print("The total time spent for extracting Recommended Ussers:                    ", str(end_extract_recommended_users_tweets_t - start_user_recommending_t).split('.')[0])
print("The total time spent for extract and show Tweets' of Recommended Users:    ", str(end_notebook_t - end_extract_recommended_users_tweets_t).split('.')[0])
print('_'*85)
print("The total time spent for this script:                                      ", str(end_notebook_t-start_notebook_t).split('.')[0])


The total time spent for showing a sample of tweets and count tweets:       0:00:42
The total time spent for extracting Recommended Ussers:                     0:00:37
The total time spent for extract and show Tweets' of Recommended Users:     0:00:29
_____________________________________________________________________________________
The total time spent for this script:                                       0:01:49
